Title: K-means clustering

Authors: Bauyrzhan Taimanov - Madina Amanatayeva

Date: Jan 18, 2024

In [8]:
# imports 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

# 0) Context


## Customer Segmentation with Parallel K-Means

In business, we often say "Customer is king" and "Customer is always right." Basically, it means we need to focus on what customers want. So, the idea here is to use this concept to group products that match what different groups of customers like. This can help a lot in online stores because it means showing customers the products they're most likely to be interested in.

![alt text](https://editor.analyticsvidhya.com/uploads/73654cluster.jpg)

This concept can be handy in places where we use digital marketing tools to promote products. For example, in some types of ads, we show products to people based on things like their interests. Then, by looking at whether they click on the ads or not, we can figure out who else might like those products.
Similarly, in other types of ads, we bid for space to show our products to people in real-time. We do this based on what we know about the people seeing the ads and what's worked well in the past. This way, we can make sure we're showing the right products to the right people at the right time.

## Concept

Customers buy products based on different needs and constraints, like budget and preference for certain items. To understand these behaviors, we can analyze customer invoice data and other non-personal data.
Here's a simplified approach:
* Analyze customer invoice data to find buying patterns.
* Use clustering algorithms to group customers based on similar purchase behaviors.
* Identify products that match each customer group's preferences.
* Tailor digital campaigns to each customer group for better targeting.

Though, with our current data limitations, we may not have all the necessary details to build a complete model.

# 1) Analysis of the Serial Algorithm

## K-Means Algorithm 

K-means clustering stands as a well-known machine learning technique utilized for customer segmentation, driven by identifying similarities among them. Its goal is to discern k clusters within the data, with each cluster denoting a cohort of customers sharing common traits. The algorithm hinges on centroids, acting as the central points of these clusters, and employs distance measures to allocate customers into the relevant clusters.


![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/K-means_convergence.gif/440px-K-means_convergence.gif)

## How does algorithm work?

The K-means algorithm is an iterative clustering method that divides a dataset into K clusters, assigning each data point to the cluster with the closest mean, or centroid. Below outlines the functioning of the algorithm:

### 1) Initialization 

 Randomly initialize K cluster centroids. These centroids represent the initial guesses for the cluster centers.

Formula: 
Random initizalization of centroids
{μ1,μ2,μ3,……..,μn}

### 2)  Data Points to Nearest Centroid:
For each data point in the dataset, calculate the distance to each centroid using a distance metric (usually Euclidean distance). Assign each data point to the cluster with the nearest centroid.

Formula: 
1) Find euclean distance between data point xi and centroid μk


![alt text](formulas/formula0.png)

2) Assign data point xi to the cluster with the nearest

![alt text](formulas/formula1.png)

### 3) Updating centroids
Calculate the mean of all data points assigned to each cluster and update the centroid to this mean.

Formula:

Mean of data points assigned to cluster k

![alt text](formulas/formula2.png)

### 4) Repeat Until Convergence

   - Convergence criteria: The algorithm iteratively performs steps 2 and 3 until convergence is achieved. Convergence occurs when either of the following conditions is met:
   - The centroids no longer change significantly between iterations. This can be determined by comparing the current centroids to the centroids from the previous iteration and checking if the change is below a predefined threshold.
   - A maximum number of iterations is reached.


### 5) Final Clustering

   - Once convergence is achieved, the final clustering is obtained, with each data point assigned to a specific cluster based on the nearest centroid.
   - The final clusters represent groups of data points that are similar to each other in terms of their features, with the centroids serving as representative points of each cluster.

The goal of the K-means algorithm is to minimize the within-cluster sum of squares (WCSS), which is the sum of the squared distances between each data point and its assigned centroid within the cluster. This objective function represents the compactness of the clusters, aiming to minimize the variance within each cluster while maximizing the separation between clusters.

It's important to note that K-means is sensitive to the initial placement of centroids, and different initializations may lead to different clustering results. Therefore, it's common to run the algorithm multiple times with different initializations and choose the clustering with the lowest WCSS as the final result. Additionally, determining the optimal number of clusters (K) is a critical step and is often done using techniques like the Elbow Method or silhouette analysis.

Our implementation of the K-Means Clustering algorithm is as follows:

It is based on the C++ code found at the following GitHub repository: https://github.com/aditya1601/kmeans-clustering-cpp

```cpp
  void run(vector<Point> &all_points)
    {
        total_points = all_points.size();
        dimensions = all_points[0].getDimensions();

        // Initializing Clusters
        vector<int> used_pointIds;

        for (int i = 1; i <= K; i++)
        {
            while (true)
            {
                int index = rand() % total_points;

                if (find(used_pointIds.begin(), used_pointIds.end(), index) ==
                    used_pointIds.end())
                {
                    used_pointIds.push_back(index);
                    all_points[index].setCluster(i);
                    Cluster cluster(i, all_points[index]);
                    clusters.push_back(cluster);
                    break;
                }
            }
        }
       cout << "Clusters initialized = " << clusters.size() << endl
             << endl;

        cout << "Running K-Means Clustering.." << endl;

        int iter = 1;
        while (true)
        {
            cout << "Iter - " << iter << "/" << iters << endl;
            bool done = true;

            // Add all points to their nearest cluster
            #pragma omp parallel for reduction(&&: done) num_threads(16)
            for (int i = 0; i < total_points; i++)
            {
                int currentClusterId = all_points[i].getCluster();
                int nearestClusterId = getNearestClusterId(all_points[i]);

                if (currentClusterId != nearestClusterId)
                {
                    all_points[i].setCluster(nearestClusterId);
                    done = false;
                }
            }

            // clear all existing clusters
            clearClusters();
             // reassign points to their new clusters
            for (int i = 0; i < total_points; i++)
            {
                // cluster index is ID-1
                clusters[all_points[i].getCluster() - 1].addPoint(all_points[i]);
            }

            // Recalculating the center of each cluster
            for (int i = 0; i < K; i++)
            {
                int ClusterSize = clusters[i].getSize();

                for (int j = 0; j < dimensions; j++)
                {
                    double sum = 0.0;
                    if (ClusterSize > 0)
                    {
                        #pragma omp parallel for reduction(+: sum) num_threads(16)
                        for (int p = 0; p < ClusterSize; p++)
                        {
                            sum += clusters[i].getPoint(p).getVal(j);
                        }
                        clusters[i].setCentroidByPos(j, sum / ClusterSize);
                    }
                }
            }
              if (done || iter >= iters)
            {
                cout << "Clustering completed in iteration : " << iter << endl
                     << endl;
                break;
            }
            iter++;
        }

        ofstream pointsFile;
        pointsFile.open(output_dir + "/" + to_string(K) + "-points.txt", ios::out);

        for (int i = 0; i < total_points; i++)
        {
            pointsFile << all_points[i].getCluster() << endl;
        }

        pointsFile.close();

        // Write cluster centers to file
        ofstream outfile;
        outfile.open(output_dir + "/" + to_string(K) + "-clusters.txt");
        if (outfile.is_open())
        {
             for (int i = 0; i < K; i++)
            {
                cout << "Cluster " << clusters[i].getId() << " centroid : ";
                for (int j = 0; j < dimensions; j++)
                {
                    cout << clusters[i].getCentroidByPos(j) << " ";    // Output to console
                    outfile << clusters[i].getCentroidByPos(j) << " "; // Output to file
                }
                cout << endl;
                outfile << endl;
            }
            outfile.close();
        }
        else
        {
            cout << "Error: Unable to write to clusters.txt";
        }
    }
};
```

# 2) A-priori study of available parallelism

## Problem

For the shop owners, whether they operate an e-commerce platform or a physical supermarket, understanding their customers is crucial, regardless of the scale of their business, be it a small local shop or a large corporation like Amazon or Netflix.
They are able to gather basic customer data from those who hold membership cards, including details such as age, annual income, and spending score. The spending score reflects customer behavior and purchasing patterns. With new products entering the market, they aim to tailor your marketing efforts to specific customer segments for each product.
Clustering, a fundamental problem in unsupervised learning, becomes essential in categorizing individuals into groups based on similarities. These groups, known as clusters, consist of data points that share more similarities within the cluster than with data points in other clusters. Distance-based clustering involves grouping data points into clusters where the distances between points within a cluster are minimized, while distances between clusters are maximized.

In [ ]:
K-means clustering algorithm to cluster customer data based on age, income, spending score, and frequency. Here's an overview of the functionalities:

Point Structure: Represents a customer with attributes such as age, income, spending score, frequency, and cluster assignment.
EuclideanDistance Function: Calculates the Euclidean distance between two customer points based on their attributes.
UpdateCentroids Function: Updates the centroids of clusters based on the mean of the points assigned to each cluster.
HasConverged Function: Checks if the centroids have converged within a specified epsilon threshold.
Kmeans Function: Implements the K-means clustering algorithm by iteratively assigning points to the nearest centroids and updating centroids until convergence.
Main Function: Reads customer data from a file, initializes centroids randomly, performs K-means clustering, and measures execution time.